In [21]:
import pandas as pd

In [22]:
file_name = "schools_complete.csv"
school_data = pd.read_csv(file_name)

# Remove the rows with missing data
school_data = school_data.dropna(how="any")

school_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [23]:
file_name = "students_complete.csv"
student_data = pd.read_csv(file_name)

# Remove the rows with missing data
student_data = student_data.dropna(how="any")

student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [24]:
total_schools = school_data['School ID'].count()

In [25]:
total_students = student_data['Student ID'].count()

In [26]:
total_budget = school_data.budget.sum()

In [27]:
avg_math_score = student_data.math_score.mean()

In [28]:
avg_reading_score = student_data.reading_score.mean()

In [29]:
num_passing_math = student_data[student_data.math_score >= 70]['Student ID'].count()
perc_passing_math = round(num_passing_math / total_students * 100,2)

In [30]:
num_passing_reading = student_data[student_data.reading_score >= 70]['Student ID'].count()
perc_passing_reading = round(num_passing_reading / total_students * 100,2)

In [31]:
overall_passing_rate = (perc_passing_math + perc_passing_reading) / 2

In [32]:
# District Summary

district_summary_df = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [perc_passing_math],
    "% Passing Reading": [perc_passing_reading],
    "Overall Passing Rate": [overall_passing_rate]
})
district_summary_df = district_summary_df.style.hide_index()
district_summary_df

Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
15,39170,24649428,78.9854,81.8778,74.98,85.81,80.395


In [63]:
merge_table = pd.merge(student_data, school_data, on="school_name")
merge_table.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [64]:
school_groupby = merge_table.groupby('school_name')

In [65]:
school_type = school_groupby.type.max()

In [66]:
total_students_by_school = school_groupby['Student ID'].count()

In [67]:
total_school_budget = school_groupby.budget.max()

In [68]:
per_student_budget = round(total_school_budget / total_students_by_school, 2)

In [69]:
avg_math_score_by_school = school_groupby.math_score.mean()

In [70]:
avg_reading_score_by_school = school_groupby.reading_score.mean()

In [71]:
num_passing_math_by_school = merge_table.loc[merge_table.math_score >= 70].groupby(["school_name"])["Student ID"].count()
perc_passing_math_by_school = round(num_passing_math_by_school / total_students_by_school * 100, 2)

In [72]:
num_passing_reading_by_school = merge_table.loc[merge_table.reading_score >= 70].groupby(["school_name"])["Student ID"].count()
perc_passing_reading_by_school = round(num_passing_reading_by_school / total_students_by_school * 100, 2)

In [73]:
overall_passing_rate_by_school = (perc_passing_math_by_school + perc_passing_reading_by_school) / 2

In [74]:
overall_summary_by_school_df = pd.DataFrame({
      "School Type": school_type,
      "Total Students": total_students_by_school,
      "Total School Budget": total_school_budget,
      "Per Student Budget": per_student_budget,
      "Average Math Score": avg_math_score_by_school,
      "Average Reading Score": avg_reading_score_by_school,
      "% Passing Math": perc_passing_math_by_school, 
      "% Passing Reading": perc_passing_reading_by_school,
      "Overall Passing Rate": overall_passing_rate_by_school
    })

overall_summary_by_school_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.68,81.93,74.305
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.13,97.04,95.585
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.99,80.74,73.365
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.31,79.30,73.805
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.39,97.14,95.265
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.75,80.86,73.805
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.51,96.25,94.380
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.68,81.32,73.500
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.06,81.22,73.640


In [75]:
# Top Performing Schools

top5_schools = overall_summary_by_school_df.sort_values("Overall Passing Rate", ascending=False).head(5)
top5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.13,97.04,95.585
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.27,97.31,95.290
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.59,95.95,95.270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.39,97.14,95.265
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.87,96.54,95.205


In [76]:
# Bottom Performing Schools

bottom5_schools = overall_summary_by_school_df.sort_values("Overall Passing Rate", ascending=False).tail(5)
bottom5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.75,80.86,73.805
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.06,81.22,73.640
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.68,81.32,73.500
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.99,80.74,73.365
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.37,80.22,73.295


In [77]:
# Math Scores by Grade

math_scores_9th = merge_table.loc[merge_table.grade=='9th'].groupby(["school_name"]).math_score.mean()
math_scores_10th = merge_table.loc[merge_table.grade=='10th'].groupby(["school_name"]).math_score.mean()
math_scores_11th = merge_table.loc[merge_table.grade=='11th'].groupby(["school_name"]).math_score.mean()
math_scores_12th = merge_table.loc[merge_table.grade=='12th'].groupby(["school_name"]).math_score.mean()

mean_math_scores = pd.DataFrame({
    "9th": math_scores_9th,
    "10th": math_scores_10th,
    "11th": math_scores_11th,
    "12th": math_scores_12th
})
mean_math_scores


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [78]:
# Reading Scores by Grade

reading_scores_9th = merge_table.loc[merge_table.grade=='9th'].groupby(["school_name"]).reading_score.mean()
reading_scores_10th = merge_table.loc[merge_table.grade=='10th'].groupby(["school_name"]).reading_score.mean()
reading_scores_11th = merge_table.loc[merge_table.grade=='11th'].groupby(["school_name"]).reading_score.mean()
reading_scores_12th = merge_table.loc[merge_table.grade=='12th'].groupby(["school_name"]).reading_score.mean()

mean_reading_scores = pd.DataFrame({
    "9th": reading_scores_9th,
    "10th": reading_scores_10th,
    "11th": reading_scores_11th,
    "12th": reading_scores_12th
})
mean_reading_scores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [82]:
bins_list = [0, 585, 615, 645, 675]
labels_list = ["<$585", "$585-$615", "$615-$645", "$645-$675"]
overall_summary_by_school_df['Spending Range'] = pd.cut(overall_summary_by_school_df['Per Student Budget'], bins=bins_list, labels=labels_list)
overall_summary_by_school_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Range
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.68,81.93,74.305,$615-$645
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.13,97.04,95.585,<$585
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.99,80.74,73.365,$615-$645
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.31,79.30,73.805,$615-$645
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.39,97.14,95.265,$615-$645


In [83]:
avg_math_score = overall_summary_by_school_df.groupby('Spending Range')['Average Math Score'].mean()
avg_reading_score = overall_summary_by_school_df.groupby('Spending Range')['Average Reading Score'].mean()
perc_passing_math = overall_summary_by_school_df.groupby('Spending Range')['% Passing Math'].mean()
perc_passing_reading = overall_summary_by_school_df.groupby('Spending Range')['% Passing Reading'].mean()
overall_passing_rate = (perc_passing_math + perc_passing_reading) / 2

In [86]:
scores_by_spending_df = pd.DataFrame ({
    "Average Math Score": avg_math_score,
    "Average Reading Score": avg_reading_score,
    "% Passing Math": perc_passing_math,
    "% Passing Reading": perc_passing_reading,
    "Overall Passing Rate": overall_passing_rate
})
scores_by_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Range,,,,,
<$585,83.455399,83.933814,93.460000,96.610000,95.035000
$585-$615,83.599686,83.885211,94.230000,95.900000,95.065000
$615-$645,79.079225,81.891436,75.668333,86.106667,80.887500
$645-$675,76.997210,81.027843,66.163333,81.133333,73.648333


In [95]:
bins_list = [0, 1000, 2000, 5000]
labels_list = ["Small (< 1000)", "Medium (1000 - 2000)", "Large (2000 - 5000)"]
overall_summary_by_school_df['School Size'] = pd.cut(overall_summary_by_school_df['Total Students'], bins=bins_list, labels=labels_list)


In [96]:
avg_math_score = overall_summary_by_school_df.groupby('School Size')['Average Math Score'].mean()
avg_reading_score = overall_summary_by_school_df.groupby('School Size')['Average Reading Score'].mean()
perc_passing_math = overall_summary_by_school_df.groupby('School Size')['% Passing Math'].mean()
perc_passing_reading = overall_summary_by_school_df.groupby('School Size')['% Passing Reading'].mean()
overall_passing_rate = (perc_passing_math + perc_passing_reading) / 2

In [97]:
scores_by_school_size_df = pd.DataFrame ({
    "Average Math Score": avg_math_score,
    "Average Reading Score": avg_reading_score,
    "% Passign Math": perc_passing_math,
    "% Passign Reading": perc_passing_reading,
    "Overall Passing Rate": overall_passing_rate
})
scores_by_school_size_df

,Average Math Score,Average Reading Score,% Passign Math,% Passign Reading,Overall Passing Rate
School Size,,,,,
Small (< 1000),83.821598,83.929843,93.55000,96.10000,94.825
Medium (1000 - 2000),83.374684,83.864438,93.59800,96.79000,95.194
Large (2000 - 5000),77.746417,81.344493,69.96375,82.76625,76.365


In [98]:
school_type_groupby = overall_summary_by_school_df.groupby('School Type')

In [99]:
avg_math_score = school_type_groupby['Average Math Score'].mean()
avg_reading_score = school_type_groupby['Average Reading Score'].mean()
perc_passing_math = school_type_groupby['% Passing Math'].mean()
perc_passing_reading = school_type_groupby['% Passing Reading'].mean()
overall_passing_rate = (perc_passing_math + perc_passing_reading) / 2

In [100]:
scores_by_school_type_df = pd.DataFrame ({
    "Average Math Score": avg_math_score,
    "Average Reading Score": avg_reading_score,
    "% Passing Math": perc_passing_math,
    "% Passing Reading": perc_passing_reading,
    "Overall Passing Rate": overall_passing_rate
})
scores_by_school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620000,96.586250,95.103125
District,76.956733,80.966636,66.548571,80.798571,73.673571
